# Coursework 2
(core codebase to be imported as modules)

In [1]:
import numpy as np
from perceptron import Perceptron, KernelPerceptron
from kernels import polynomial_kernel, radial_basis_kernel

In [2]:
# Loading Data
data = np.loadtxt("dtrain123.dat")
y = data[:, 0]
X = data[:, 1:]

In [3]:
# Build test XOR dataset
a = np.array([[1,1], [-1,-1], [-1,1], [1,-1]])
b = np.array([1,1,-1,-1])

In [4]:
# Run for regular unkernalised perceptron
p = Perceptron()
p.train(a, b)
p.predict(a)

array([ 0.,  0.,  0.,  0.])

In [5]:
# Run for 2-order polynomial kernel
p_2 = KernelPerceptron(polynomial_kernel, 2)
p_2.train(a,b)
p_2.predict_proba(a)

array([ 4.,  4., -8., -8.])

In [6]:
p_2.gram

array([[4, 4, 0, 0],
       [4, 4, 0, 0],
       [0, 0, 4, 4],
       [0, 0, 4, 4]])

## Test for differentiating between 1s and 3s

In [7]:
idx = y!=2
X_z = X[idx,:]
y_z = y[idx]
y_z = (y_z == 1).astype(int) - (y_z == 3).astype(int)
n_1 = sum(y==1)
n_3 = sum(y==3)
print(n_1, n_3)

120 70


### Regular perceptron

In [8]:
p = Perceptron()
p.train(X_z, y_z)

from utilities import preprocessing, train_test_split,y_encode,data_split

In [3]:
data = np.loadtxt("zipcombo.dat")

This will be a one vs all scenario.

In [18]:
# first class is zero

# matrix to hold predictions


X,y = data_split(data,y_col=0)
X_train,X_test,y_train,y_test = train_test_split(X,y)

predictions = np.zeros((X_test.shape[0],10))


y_train_i = y_encode(y_train,0)

P = Perceptron() # quadratic kernel
P.train(X_train,y_train_i)
print(P.M)
predictions[:,0] = P.predict_proba(X_test)

[209.]


In [19]:
for i in range(1,10):
    y_train_i = y_encode(y_train,i)
    print(y_train_i[0:10])
    P = Perceptron() # quadratic kernel
    print(P.M)
    P.train(X_train,y_train_i)
    print(P.M)
    predictions[:,i] = P.predict_proba(X_test)

[-1  1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[97.]
[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1]
0
[244.]
[-1 -1 -1 -1 -1 -1 -1  1 -1 -1]
0
[286.]
[-1 -1 -1  1  1 -1 -1 -1 -1 -1]
0
[301.]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[302.]
[-1 -1 -1 -1 -1 -1  1 -1 -1 -1]
0
[187.]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[214.]
[-1 -1  1 -1 -1 -1 -1 -1 -1 -1]
0
[314.]
[-1 -1 -1 -1 -1 -1 -1 -1  1 -1]
0
[321.]


In [20]:
preds = np.argmax(predictions,axis=1)

In [9]:
preds.shape

(3100,)

In [21]:
p.predict(X_z).dot(y_z)/len(y_z)

1.0

In [10]:
p.predict(X_z).dot(y_z)

190.0

In [11]:
X_z.shape

(190, 256)

In [12]:
### Polynomial Kernel
for d in range(1,5):
    p_k = KernelPerceptron(kernel=polynomial_kernel, k_params=d)
    p_k.train(X_z, y_z)
    error = p_k.predict(X_z).dot(y_z)/len(y_z)
    print("{} : {}".format(d, error))

1 : 0.6842105263157895
2 : 0.8526315789473684
3 : 0.9263157894736842
4 : 1.0


In [22]:
y_test[1:20]

array([7., 0., 1., 5., 1., 6., 6., 0., 7., 6., 0., 9., 8., 6., 0., 2., 4.,
       4., 0.])